In [158]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [18]:
#GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [93]:
# Create the data
a = np.round(  np.linspace(-10,10,num=10000)  )
np.random.shuffle(a)

b = np.round(  np.linspace(-10,10,num=10000)  )
np.random.shuffle(b)

sumAB = np.add(a, b)

In [125]:
# Normalize the data
aNorm    = a / np.max(a)
bNorm    = b / np.max(b)
sumNorm  = sumAB / np.max(sumAB)
dataNorm = np.vstack((aNorm, bNorm)).T

In [127]:
dataNorm.shape

(10000, 2)

In [128]:
sumNorm.shape

(10000,)

In [131]:
# Step1: Convert numpy data into tensor data
x = torch.tensor( dataNorm ).short()
y  = torch.tensor( sumNorm,  ).short()

In [148]:
# Step2: use scikit learn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(x, y, test_size=0.1)

In [149]:
# Step 3: Convert into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data  = TensorDataset(test_data , test_labels)

In [155]:
train_data.tensors[0].shape

torch.Size([9000, 2])

In [156]:
test_data.tensors[0].shape

torch.Size([1000, 2])

In [150]:
# Step 4: Translate into dataloader objects
batchsize    = 32
train_loader = DataLoader(train_data, batch_size=batchsize, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_data,  batch_size=test_data.tensors[0].shape[0])

In [160]:
# Create the Deep Learning Model
def createTheSumABNet():
    class sumABNet(nn.Module):
        def __init__(self):
            super().__init__()
            
            self.input = nn.Linear(2,20)
            
            self.bnormHidden =  nn.BatchNorm1d(20)
            self.hidden1 = nn.Linear(20,10)
            
            self.output = nn.Linear(10,1)
            
        def forward(self, x):
            
            x = F.leaky_relu( self.input(x) )
            
            x = self.bnormHidden(x)
            x = F.leaky_relu( self.hidden1(x) )
                        
            x = self.output(x)
            return x
        
    sumNet = sumABNet()
    
    lossfun = nn.MSELoss()
    
    optimizer = torch.optim.Adam(sumNet.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)
    
    return sumNet, lossfun, optimizer


        